In [2]:
from typing import NewType

import torch
import numpy as np
import pandas as pd

from torch.utils.data import Dataset, DataLoader
import torch.nn as nn                 
import torch.nn.functional as F           
import torch.optim as optim

In [3]:
# Constants

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
lr = 0.1
batch_size = 10
epochs = 100
momentum = 0.9

In [4]:
# Loading the data

df = pd.read_csv('iris-dataset.csv')
print(df)

test = df.to_numpy()

y = test[:,4]
print(y.shape)

X = test[:,:4]
print(X.shape) ### Problem is that I need to create test and training sets for both X and y...




     sepal_length  sepal_width  petal_length  petal_width    species
0             5.1          3.5           1.4          0.2     setosa
1             4.9          3.0           1.4          0.2     setosa
2             4.7          3.2           1.3          0.2     setosa
3             4.6          3.1           1.5          0.2     setosa
4             5.0          3.6           1.4          0.2     setosa
..            ...          ...           ...          ...        ...
145           6.7          3.0           5.2          2.3  virginica
146           6.3          2.5           5.0          1.9  virginica
147           6.5          3.0           5.2          2.0  virginica
148           6.2          3.4           5.4          2.3  virginica
149           5.9          3.0           5.1          1.8  virginica

[150 rows x 5 columns]
(150,)
(150, 4)


In [5]:
#Loading the data using sk-learn

from sklearn.datasets import load_iris

iris=load_iris()

for keys in iris.keys() :
    print(keys)

data
target
frame
target_names
DESCR
feature_names
filename
data_module


In [6]:
X=iris.data
y=iris.target
print(y)

y = iris['target']
print(y)

print(f'target names : {iris.target_names}\n')

print(type(X))
print(type(y))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
target names : ['setosa' 'versicolor' 'virginica']

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [7]:
# Splitting the training and the testing data

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

print(f'X_train shape : {X_train.shape}')
print(f'X_test shape : {X_test.shape}')
print(f'y_train shape : {y_train.shape}')
print(f'y_test shape : {y_test.shape}')

print(y_test) # It does contain all the three types to my surprise

X_train shape : (120, 4)
X_test shape : (30, 4)
y_train shape : (120,)
y_test shape : (30,)
[1 2 1 0 2 2 1 0 0 1 1 1 2 0 1 0 2 1 1 2 1 2 2 0 2 2 1 0 0 0]


In [8]:
# Making it compatible with PyTorch

X_train, y_train, X_test, y_test = map(
    torch.tensor, (X_train, y_train, X_test, y_test)
)

In [9]:
print(type(X_train),type(X_test))
print(type(y_train),type(y_test))
print(y_train.min(), y_train.max())

<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor(0) tensor(2)


In [10]:
# Using TensorDataset and DataLoader

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

train_ds = TensorDataset(X_train, y_train)
train_dl = DataLoader(train_ds, batch_size)

In [11]:
### GET THE DATA:

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

def load_data(test_size=0.2):
    iris=load_iris()
    X=iris.data
    y=iris.target
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=test_size)
    X_train, y_train, X_test, y_test = map(
    torch.tensor, (X_train, y_train, X_test, y_test)
)
    return X_train,X_test,y_train,y_test

def get_data(train_ds : TensorDataset, test_ds : TensorDataset, batch_size : int):
    return (
        DataLoader(train_ds, batch_size, shuffle=True),
        DataLoader(test_ds, batch_size=batch_size * 2),
    )
    
    

In [12]:
# Neural network using nn.Module


import torch.nn as nn    

class IrisClassifier(nn.Module):
    
    def __init__(self):
        super(IrisClassifier, self).__init__()
        self.layer = nn.Linear(4, 3)
        
    def forward(self, x):
        return F.softmax(self.layer(x))


In [13]:
my_model = IrisClassifier()

print(my_model)

IrisClassifier(
  (layer): Linear(in_features=4, out_features=3, bias=True)
)


In [14]:
loss_function = nn.CrossEntropyLoss()

optimizer = optim.SGD(my_model.parameters(), lr=0.01, momentum=0.9)

In [15]:
### Get the model

def get_model_with_optimizer(lr, momentum):
    model = IrisClassifier()
    return model, optim.SGD(model.parameters(), lr=lr, momentum=momentum)



In [16]:
### Coding the training loop i.e. the fit() function

def fit(model,opt,epochs,loss_function, train_dl, test_dl):
    for epoch in range(epochs):
        model.train()
        for x_training_tensor, y_label in train_dl:
            prediction = model(x_training_tensor.float())
            loss = loss_function(prediction, y_label)

            loss.backward()
            opt.step()
            opt.zero_grad()

        model.eval()
        with torch.no_grad():
            valid_loss = sum(loss_function(model(x_training_tensor.float()), y_label) for x_training_tensor, y_label in test_dl)

        print(epoch, valid_loss)

In [17]:
epochs = 100

X_train,X_test,y_train,y_test = load_data()
train_ds = TensorDataset(X_train, y_train)
test_ds = TensorDataset(X_test, y_test)
train_dl, test_dl = get_data(train_ds, test_ds, batch_size)



model, opt = get_model_with_optimizer(lr, momentum)
loss_function = nn.CrossEntropyLoss()

for x,y in train_dl:
    #print(f'x : {x}\n y : {y}\n')
    output = model(x.float())
    #print(f'x shape : {x.shape}\n')
    #print(f'y shape : {y.shape}\n')
    #print(f'output shape : {output.shape}\n')
    #print(f'output max min : {output.max()} , {output.min()}\n')
    #print(f'y max min : {y.max()} , {y.min()}\n')
    #prediction = output.max(1, keepdim=False)[1]
    prediction = torch.max(output, 1)[1]
    #print(f'prediction shape : {prediction.shape}\n')
    print(prediction == y)
    print(prediction)
    #print(f'sum(prediction == y) : {sum(prediction == y)}\n')
    loss = loss_function(output, y)
    print(f'loss : {loss}\n')
    print(f'loss.item() : {loss.item()}\n')
    


#fit(model,opt,epochs,loss_function, train_dl, test_dl)



tensor([False,  True, False, False, False, False, False, False, False, False])
tensor([1, 1, 2, 2, 1, 2, 2, 2, 2, 1])
loss : 1.1680114269256592

loss.item() : 1.1680114269256592

tensor([False, False, False, False, False, False,  True, False, False, False])
tensor([2, 2, 2, 2, 1, 2, 1, 2, 2, 2])
loss : 1.149756669998169

loss.item() : 1.149756669998169

tensor([False, False, False, False, False, False, False, False, False, False])
tensor([1, 1, 1, 1, 1, 2, 2, 2, 1, 1])
loss : 1.1618506908416748

loss.item() : 1.1618506908416748

tensor([False, False, False, False, False,  True, False, False, False, False])
tensor([1, 2, 2, 2, 2, 1, 1, 1, 2, 1])
loss : 1.173966646194458

loss.item() : 1.173966646194458

tensor([False, False,  True,  True, False, False, False, False, False, False])
tensor([2, 1, 1, 1, 1, 1, 2, 2, 1, 1])
loss : 1.1400690078735352

loss.item() : 1.1400690078735352

tensor([False, False, False, False, False, False, False, False, False, False])
tensor([2, 2, 2, 2, 2, 2, 2, 1

/var/folders/xd/10nyn_j13qs58rww11sjkgy80000gn/T/ipykernel_1079/1432828102.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(self.layer(x))
